In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium import plugins
import pickle
from scipy.ndimage import imread
import gdal, ogr, osr
import numpy as np

In [2]:
spray = pd.read_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/assets/spray.csv')
train = pd.read_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/assets/train.csv')
test = pd.read_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/assets/test.csv')

In [3]:
testsunique = test.nunique()

In [4]:
spray.drop('Time',axis = 1, inplace=True)

In [5]:
spray['Date'] = pd.to_datetime(spray['Date'], format='%Y/%m/%d')

In [6]:
spray['Date'] = pd.to_datetime(spray['Date'], format='%Y/%m/%d')

#2011: 08-29 to 09/07
#2013: 07-17 to 09/05

spray2011 = spray.loc[(spray['Date'] > '2011-08-29') & (spray['Date'] <= '2011-09-07')]
spray2013 = spray.loc[(spray['Date'] > '2013-07-17') & (spray['Date'] <= '2013-09-05')]

spray2011 = spray2011.reset_index(drop=True)
spray2013 = spray2013.reset_index(drop=True)

#Finding spray dates within the years given

In [7]:
TrapUnique = train.drop_duplicates(['Trap'])

In [8]:
TrapUnique = TrapUnique.reset_index(drop = True)

In [5]:
TrapUnique.to_csv('/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/TrapUnique.csv')

In [9]:
TrapUnique['AddressNumberAndStreet'] = TrapUnique['AddressNumberAndStreet'].str.replace(', Chicago, IL',  '')

In [10]:
TrapUnique['CombinedAddress'] = TrapUnique['Trap']+', ' + TrapUnique['AddressNumberAndStreet']

In [11]:
TrapLocations = TrapUnique[['Latitude', 'Longitude']]
TrapLocationsList = TrapLocations.values.tolist()

TrapLabels = TrapUnique["CombinedAddress"].values.tolist()

#This cell is setting the labels for each of the markers. When clicked, the marker will display the Street Address and TrapID

In [12]:
ZoneMap = folium.Map(location = [41.883808, -87.632296], zoom_start= 10, control_scale=True)

In [59]:
#!!!WARNING!!! DO NOT RUN THIS CELL. Doing so will rerun the mapping command, and you will be waiting a while.
from folium.plugins import MarkerCluster
from folium import IFrame

ZoneMap = folium.Map(location = [41.883808, -87.632296], zoom_start= 11, control_scale=True)

#City Limits
CityLimits = folium.FeatureGroup(name='City Limits')
CityLimits.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/ChicagoLimits.geojson'))
ZoneMap.add_child(CityLimits)

#Adding in the different Zones
Zones = folium.FeatureGroup(name = 'Zones')

Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Business.geojson', style_function=lambda x: {'fillColor' : '#ff0000', 'color' : '#ff0000'}, name='Business'))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Commercial.geojson', style_function=lambda x: {'fillColor' : '#ff9400', 'color' : '#ff9400'}, name='Commercial').add_to(Zones))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Manufacturing.geojson', style_function=lambda x: {'fillColor' : '#fffa00', 'color' : '#fffa00'}, name='Manufacturing').add_to(Zones))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Residential.geojson', style_function=lambda x: {'fillColor' : '#72ff00', 'color' : '#72ff00'}, name='Residential').add_to(Zones))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Developments.geojson', style_function=lambda x: {'fillColor' : '#00faff', 'color' : '#00faff'}, name='Developments').add_to(Zones))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/Downtown.geojson', style_function=lambda x: {'fillColor' : '#b302f9', 'color' : '#b302f9'}, name='Downtown').add_to(Zones))
Zones.add_child(folium.GeoJson(data='/Users/skiparay/Desktop/GA/Projects/project-4/GA-West_Nile_Team_3/Steve\'s GIS Shenanigans/Zone Files/PublicParks.geojson', style_function=lambda x: {'fillColor' : '#f901aa', 'color' : '#f901aa'}, name='Public Parks').add_to(Zones))

ZoneMap.add_child(Zones)

#Marker Clusters for 2011 Spraying Locations  
locations2011 = []
for index,row in spray2011.iterrows():
    locations2011.append([row['Latitude'], row['Longitude']])                           
                        
s2011 = folium.FeatureGroup(name='2011 Spray Locations')
s2011.add_child(MarkerCluster(locations=locations2011, overlay=True))
ZoneMap.add_child(s2011)

#Marker Clusters for 2013 Spraying Locations  
locations2013 = []
for index,row in spray2013.iterrows():
    locations2013.append([row['Latitude'], row['Longitude']])
                          
s2013 = folium.FeatureGroup(name='2013 Spray Locations')
s2013.add_child(MarkerCluster(locations=locations2013, overlay=True))
ZoneMap.add_child(s2013)

#Trap Locations
TrapMarker = folium.FeatureGroup(name = 'Traps')
for i in range(0,len(TrapUnique)):
    TrapMarker.add_child(folium.Marker([TrapUnique.iloc[i]['Latitude'], TrapUnique.iloc[i]['Longitude']], popup=TrapUnique.iloc[i]['CombinedAddress']))   
ZoneMap.add_child(TrapMarker)

ZoneMap.add_child(folium.map.LayerControl())
ZoneMap.save('Chicago')
